In [2]:
%run ../scripts/notebook_settings_lean.py
import tskit

In [ ]:
ts = tskit.load("../steps/All_Samples_relate/chromX_tskit.trees")

In [ ]:
ts

In [ ]:
# Version which counts number of samples, and notes both the first coalescence and when 50 % has coalesced.
def coalescence_ordering(tree, IDs, sample_counts):
    df_list = []
    for i in IDs:
        pop_list = []
        gen_list = []
        coal_counts = {}
        for p in sample_counts.index:
            coal_counts[p] = 0
        current_node = i
        while tree.depth(current_node) > 0:
            # Find parent node
            parent_node = tree.parent(current_node)
            # Determine children, and then pick alternate
            # cannot find a method for this, so I use an explicit if/else
            children = tree.children(parent_node)
            if current_node == children[0]:
                alt_node = children[1]
            else:
                alt_node = children[0]
            # Determine which populations are present under the alternate node
            alt_samples = pd.Series([x for x in tree.samples(alt_node)])
            alt_sample_counts = alt_samples.map(i_mapping).value_counts()
            for p in alt_sample_counts.index:
                coal_counts[p] += alt_sample_counts[p]
            # If pop is not already added to list, add pop and note coal time
            for p in alt_sample_counts.index:
                if p  not in pop_list and coal_counts[p] > sample_counts[p]/2:
                    pop_list.append(p)
                    gen_list.append(tree.time(current_node))
            current_node = parent_node
        d = {"ID": i, "sites": tree.num_sites, "span": tree.span, "start": tree.interval[0]}
        for i in range(len(pop_list)):
            d["coal_{}".format(i)] = pop_list[i]
        for i in range(len(gen_list)):
            d["coal_date_{}".format(i)] = gen_list[i]
        df_list.append(pd.DataFrame(d, index=[i]))
    return pd.concat(df_list)

Trying a couple of things shown in the tutorial (https://tskit.dev/tutorials/getting_started.html )

In [ ]:
ts_subset = ts.keep_intervals([[10000000, 11000000]]) #([[67000000, 69000000]]) #([[67000000, 72000000]])

In [ ]:
ts_subset

In [ ]:
# Setup based on poplabels
poplabels = pd.read_csv("../data/pops/all_females_8cluster.sample", sep=" ",
                        names=["ID", "POP", "GROUP", "SEX"], header=0)

i_mapping = {}
for i, row in poplabels.iterrows():
    i_mapping[i*2] = row.GROUP
    i_mapping[i*2+1] = row.GROUP
poplabel_counts = poplabels["GROUP"].value_counts().to_dict()
highest_ID = poplabels.index.max()*2+1

In [ ]:
poplabels.POP.value_counts()

In [ ]:
c = 0
ID_list = list(i_mapping.keys())
sample_counts = poplabels["GROUP"].value_counts()*2
df_list = []
for tree in ts_subset.trees():
    df = coalescence_ordering(tree, ID_list, sample_counts)
    if c % 1000 == 0:
        print(c)
    c += 1
    df_list.append(df)
test_df = pd.concat(df_list)

In [ ]:
test_df

In [ ]:
g = sns.histplot(data=test_df, x="sites")
g.set(xlim=(0,30))

In [ ]:
first_coal_l = []
ID_l = []
for ID in test_df.ID.unique():
    ID_df = test_df.loc[(test_df.ID == ID) & (test_df.coal_date_0 != test_df.coal_date_1)]
    first_coal_l.append(ID_df.coal_0.value_counts().index[0])
    ID_l.append(ID)

In [ ]:
poplabels

In [ ]:
check_df = pd.DataFrame({"first_coal": first_coal_l, "PDGP_ID": ID_l, "POP": pd.Series(ID_l).map(i_mapping)})

In [ ]:
for c in check_df.POP.unique():
    sub_df = check_df.loc[check_df.POP == c]
    print(c)
    print(sub_df["first_coal"].value_counts())

Doing the check with the full chromosome, which I thought had been calculated incorrectly. It was calculated incorrectly in regards to index IDs, which I have fixed now.

In [ ]:
# Setup based on poplabels
poplabels = pd.read_csv("../data/pops/all_females_8cluster.sample", sep=" ",
                        names=["ID", "POP", "GROUP", "SEX"], header=0)

i_mapping = {}
for i, row in poplabels.iterrows():
    i_mapping[i*2] = row.GROUP
    i_mapping[i*2+1] = row.GROUP
poplabel_counts = poplabels["GROUP"].value_counts().to_dict()
highest_ID = poplabels.index.max()*2+1

In [ ]:
df_l = []
for ID in poplabels.ID:
    df = (pd.read_csv("../steps/relate_coal_ordering/{}.txt".format(ID)))
    df["PDGP_ID"] = ID
    df_l.append(df)
coal_df = pd.concat(df_l)

In [ ]:
ID_list = poplabels.ID
ID_t1 = poplabels.loc[poplabels.ID == "PD_0792"].index.values[0]*2
ID_t1

In [ ]:
df_l[95]

In [ ]:
coal_df

Percentage of first coals which are order informative (only coals with that group)

In [ ]:
len(coal_df.loc[(coal_df.ID == ID) & (coal_df.coal_date_0 != coal_df.coal_date_1)])/len(coal_df)

In [ ]:
len(coal_df.PDGP_ID.unique())

In [ ]:
first_coal_l = []
ID_l = []
for ID in coal_df.PDGP_ID.unique():
    print(ID)
    ID_df = coal_df.loc[(coal_df.PDGP_ID == ID) & (coal_df.coal_date_0 != coal_df.coal_date_1)]
    first_coal_l.append(ID_df.coal_0.value_counts().index[0])
    ID_l.append(ID)
first_coal_df = pd.DataFrame({"first_coal": first_coal_l, "PDGP_ID": ID_l,
                              "POP": pd.Series(ID_l).map(dict(zip(poplabels.ID, poplabels.POP)))})

In [ ]:
first_coal_df = pd.DataFrame({"first_coal": first_coal_l, "PDGP_ID": ID_l,
                              "POP": pd.Series(ID_l).map(dict(zip(poplabels.ID, poplabels.POP)))})

In [ ]:
first_coal_df

In [ ]:
for c in first_coal_df.POP.unique():
    sub_df = first_coal_df.loc[first_coal_df.POP == c]
    print(c)
    print(sub_df["first_coal"].value_counts())

In [ ]:
poplabels.POP.value_counts()

Checking coal times and spans of trees

In [ ]:
tree_stats_df = coal_df.loc[(coal_df.PDGP_ID == "Sci_16098")]
sns.jointplot(data=tree_stats_df, x="sites", y="span")

In [ ]:
tree_stats_df = coal_df.loc[(coal_df.PDGP_ID == "Sci_16098") & (coal_df.span < 50000)]
sns.jointplot(data=tree_stats_df, x="sites", y="span")

In [ ]:
eastern_ids = poplabels.loc[poplabels["POP"] == "Eastern Yellow"].ID

In [ ]:
eastern_ids[5:6]

In [ ]:
eastern_yellow_coals = coal_df.loc[(coal_df.PDGP_ID == "PD_0224") & (coal_df.sites > 9)]# coal_df.loc[(coal_df.PDGP_ID.isin(eastern_ids)) & (coal_df.sites > 9)]
eastern_yellow_coals

In [ ]:
coal_type_l, coal_date_l, start_pos, haplo_l  = [], [], [], []
for coal_type in ["Kindae", "Hamadryas", "Eastern Yellow"]:
    for coal in range(0, 8):
        print(coal)
        coal_timings = eastern_yellow_coals.loc[eastern_yellow_coals["coal_{}".format(coal)] == coal_type]
        coal_type_l.extend(list(coal_timings["coal_{}".format(coal)]))
        coal_date_l.extend(list(coal_timings["coal_date_{}".format(coal)]))
        start_pos.extend(list(coal_timings.start))
        haplo_l.extend(list(coal_timings.ID))

In [ ]:
sub_coal_df = pd.DataFrame({"coal_type": coal_type_l, "coal_date": coal_date_l, "start": start_pos, "haplotype": haplo_l})

In [ ]:
g = sns.histplot(sub_coal_df, x="coal_date", hue="coal_type")
g.set(xlim= (-10, 70000))

In [ ]:
g = sns.histplot(sub_coal_df.loc[sub_coal_df.coal_type.isin(["Kindae", "Hamadryas"])],
                 x="coal_date", hue="coal_type")
g.set(xlim= (-10, 70000))

In [ ]:
hamadryas_coal = sub_coal_df.loc[sub_coal_df.coal_type == "Hamadryas"]

In [ ]:
len(hamadryas_coal.loc[hamadryas_coal.coal_date > 10000])/len(hamadryas_coal)

In [ ]:
hamadryas_coal.coal_date.mean()

In [ ]:
sub_coal_df

In [ ]:
coal_type_l, coal_date_l, start_pos, haplo_l  = [], [], [], []
df_l = []
for coal_type in ["Kindae", "Hamadryas"]:
    for coal in range(0, 8):
        print(coal)
        coal_timings = eastern_yellow_coals.loc[eastern_yellow_coals["coal_{}".format(coal)] == coal_type]
        coal_type_l.extend(list(coal_timings["coal_{}".format(coal)]))
        coal_date_l.extend(list(coal_timings["coal_date_{}".format(coal)]))
        start_pos.extend(list(coal_timings.start))
        haplo_l.extend(list(coal_timings.ID))
    df_l.append(pd.DataFrame({"coal_date_{}".format(coal_type): coal_date_l,
                                "start": start_pos, "haplotype": haplo_l}))
coal_comp_df = df_l[0]
for d in df_l[1:]:
    coal_comp_df = coal_comp_df.merge(d, how="left", on=["start", "haplotype"])

In [ ]:
coal_comp_df

In [ ]:
coal_comp_df.loc[coal_comp_df.coal_date_Kindae > coal_comp_df.coal_date_Hamadryas]

In [ ]:
coal_comp_df.loc[coal_comp_df.coal_date_Kindae < coal_comp_df.coal_date_Hamadryas]

In [ ]:
coal_comp_df.loc[coal_comp_df.coal_date_Kindae == coal_comp_df.coal_date_Hamadryas]

In [ ]:
g = sns.histplot(coal_comp_df.loc[coal_comp_df.coal_date_Kindae == coal_comp_df.coal_date_Hamadryas],
                 x="coal_date_Kindae")
g.set(xlim= (-10, 70000))